In [313]:
import numpy as np
import h5py
import pandas as pd 
import matplotlib.pyplot as plt

import math

import sklearn
import statsmodels
from statsmodels import robust

from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn import datasets
from scipy import stats
from sklearn.preprocessing import scale
from sklearn.covariance import EmpiricalCovariance, MinCovDet
    
## Definition of a Gaussian
def gaussian(x, mu, sig):
    return 1./(math.sqrt(2.*math.pi)*sig)*np.exp(-np.power((x - mu)/sig, 2.)/2)

## Removing Outliers Using the Median Absolute Deviation 
def remove_outliers(data):
    filtered = []
    m = 3
    u = np.median(data)
    s = robust.mad(data, c = 1, axis = 0 )
    print u
    print s
    for e in range(data.shape[0]):
        if u-3*s < data[e] < u+3*s:
            filtered.append(data[e])
        else:
            filtered.append(np.nan)
    return filtered

## Returning a 2-D Array given a 4-D Array
def dimension_reduction(eegdata, c, d):
    temp = eegdata[:,:,c,d]
    return temp

## Robust PCA Code
def robustPCA(eeg_data, method):
    if method == "RobustPCA":
        out = ""
        out += "<h3> RUNNING ROBUST PCA </h3>"
        for p in range(eeg_data.shape[3]):
            print 'Noise reduction for patient ' + str(p)
            for t in range(eeg_data.shape[2]):
                print 'Noise reduction for trial ' + str(t)
                out += '<h4>Actions for patient ' + str(p) + '</h4>'

                ##Creating Covariance Matrix and via Median Screening of Outliers
                
                new_data = []

                for c in range(eeg_data.shape[1]):
                    if new_data == []:
                        temp = remove_outliers(eeg_data[:,c,t,p])
                        new_data = temp
                        print new_data
                    else:
                        temp = remove_outliers(eeg_data[:,c,t,p])
                        new_data = np.vstack((new_data, temp))
                        print new_data
                print 1000000
                maskedarr = np.ma.array(new_data, mask=np.isnan(new_data))
                cov_mat = np.ma.cov(maskedarr,rowvar=False,allow_masked=True)
                print cov_mat

                ##Defining Eigenvalues and Eigenvectors

                eig_val_cov, eig_vec_cov = np.linalg.eig(cov_mat)
                numTimesteps = len(eeg_data[:,0,t,p])
                numElectrodes = len(eeg_data[0,:,t,p])
                for i in range(len(eig_val_cov)):
                    eigvec_cov = eig_vec_cov[:,i].reshape(1,numTimesteps).T
                    print('Eigenvector {}: \n{}'.format(i+1, eigvec_cov))
                    print('Eigenvalue {} from covariance matrix: {}'.format(i+1, eig_val_cov[i]))
                eig_pairs = [(np.abs(eig_val_cov[i]), eig_vec_cov[:,i]) for i in range(len(eig_val_cov))]
                eig_pairs.sort(key=lambda x: x[0], reverse=True)
                
                ## Making the Elbow

                array = []
                for x in range(numTimesteps):
                    array.append(eig_pairs[x][0])
                array2 = [x * 1000 for x in array] 

                p1 = len(array2)
                totalSum = [0]*(p1-1)

                for q in range(1,p1):
                    FirstArray, SecondArray = np.split(array2, [q,])
                    mu1 = np.mean(FirstArray)
                    mu2 = np.mean(SecondArray)
                    s1 = np.var(FirstArray)
                    s2 = np.var(SecondArray)
                    if q-1 == 0:
                        s1 = 0
                    totalvariance = (((q-1)*(s1*s1)) + ((p1-q-1)*(s2*s2))) / (p1-2)
                    Sum1 = 0
                    Sum2 = 0
                    for i in range(len(FirstArray)):
                        x = FirstArray[i]
                        x1 = np.log10(gaussian(x, mu1, totalvariance))
                        Sum1 += x1
                    for j in range(len(SecondArray)):
                        y = SecondArray[j]
                        y1 = np.log10(gaussian(y, mu2, totalvariance))
                        Sum2 += y1
                    totalSum[q-1] = Sum1 + Sum2

                dimension = np.argmax(totalSum) + 1
                print "The Number of Principal Components to Retain:"
                print dimension
                
                ## Making the matrix with only the wanted eigenvectors
                
                matrix_w = eig_pairs[0][1].reshape(numTimesteps,1)
                for x in range(0, dimension-1):
                    matrix_w = np.hstack((matrix_w, eig_pairs[x][1].reshape(numTimesteps,1)))
                    print matrix_w
                dotArray = dimension_reduction(eeg_data,t,p)
                eeg_data = matrix_w.T.dot(dotArray)
                print eeg_data

        return eeg_data, out
    else:
        return eeg_data, '<h3> No Robust PCA was done </h3>'


[[[[  1.   1.]
   [  1.   1.]]

  [[  1.   1.]
   [  1.   1.]]

  [[  4.   1.]
   [  4.   1.]]

  [[  2.   1.]
   [  8.   1.]]]


 [[[  2.  50.]
   [ 50.  50.]]

  [[  2.   2.]
   [  2.   2.]]

  [[  6.   1.]
   [  7.   1.]]

  [[  9.   1.]
   [  6.   1.]]]


 [[[  6.  40.]
   [ 40.  40.]]

  [[  3.   3.]
   [  3.   3.]]

  [[ 16.   1.]
   [ 18.   1.]]

  [[  5.   1.]
   [  5.   1.]]]]
(3L, 4L, 2L, 2L)
1.0
2.0
6.0
2.0
2.0
6.0
9.0
1.0
1.0
1.0
1.0


In [314]:
print eeg_sample_data.shape
print dimension_reduction(eeg_sample_data,0 ,0)
robustPCA(eeg_sample_data, "RobustPCA")

(3L, 4L, 2L, 2L)
[[  1.   1.   4.   2.]
 [  2.   2.   6.   9.]
 [  6.   3.  16.   5.]]
Noise reduction for patient 0
Noise reduction for trial 0
2.0
1.0
[1.0, 2.0, nan]
2.0
1.0
[[  1.   2.  nan]
 [  1.   2.   3.]]
6.0
2.0
[[  1.   2.  nan]
 [  1.   2.   3.]
 [  4.   6.  nan]]
5.0
3.0
[[  1.   2.  nan]
 [  1.   2.   3.]
 [  4.   6.  nan]
 [  2.   9.   5.]]
1000000
[[2.0 2.6666666666666665 1.0]
 [2.6666666666666665 11.583333333333334 7.0]
 [1.0 7.0 2.0]]
3
4
3939393933939281812783
Eigenvector 1: 
[[0.19995685274214967]
 [0.8685897012251665]
 [0.4533973841643005]]
Eigenvalue 1 from covariance matrix: 15.8511711374
Eigenvector 2: 
[[0.9736685109427075]
 [-0.12444270613066595]
 [-0.19100692052773233]]
Eigenvalue 2 from covariance matrix: 1.4630060118
Eigenvector 3: 
[[0.10948464659515142]
 [-0.4796518985852682]
 [0.8706016128766833]]
Eigenvalue 3 from covariance matrix: -1.73084381589
1
3999393939339393939
[[0.19995685274214967]
 [0.8685897012251665]
 [0.4533973841643005]]
10000000000000000

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


IndexError: too many indices for array